In [1]:
from config import *

import gensim

import glob
import os
import re

import numpy as np
import pandas as pd

import json

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

from gensim.models import ldaseqmodel

/user/ms5941/.local/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
THEME = 'GDP'

In [4]:
time_slices = pd.read_csv('Summary Stat Tables/%s_Article_Count.csv' % THEME, index_col=0)
time_slices.sort_index(inplace=True)
time_slices.index = pd.to_datetime(time_slices.index)
# time_slices.groupby(time_slices.index.year)['No. of Volatility Articles'].sum()

yearly_slices = time_slices.groupby(time_slices.index.year)['No. of %s Articles' % THEME].sum().values[:-4]

yearly_slices.sum()

In [9]:
# Load dictionary and corpus
dictionary_all = gensim.corpora.Dictionary.load(TEMP_PATH + '/%s/%s_clean.dict' % (THEME, THEME))
corpus_all = gensim.corpora.MmCorpus(TEMP_PATH + '/%s/%s_clean.mm' % (THEME, THEME))

2021-04-25 19:39:27,206 : INFO : loading Dictionary object from /work/ms5941/NLP/Temp/GDP/GDP_clean.dict
2021-04-25 19:39:27,210 : INFO : Dictionary lifecycle event {'fname': '/work/ms5941/NLP/Temp/GDP/GDP_clean.dict', 'datetime': '2021-04-25T19:39:27.210194', 'gensim': '4.0.1', 'python': '3.7.7 (default, May  7 2020, 21:25:33) \n[GCC 7.3.0]', 'platform': 'Linux-4.19.0-16-amd64-x86_64-with-debian-10.9', 'event': 'loaded'}
2021-04-25 19:39:27,466 : INFO : loaded corpus index from /work/ms5941/NLP/Temp/GDP/GDP_clean.mm.index
2021-04-25 19:39:27,469 : INFO : initializing cython corpus reader from /work/ms5941/NLP/Temp/GDP/GDP_clean.mm
2021-04-25 19:39:27,471 : INFO : accepted corpus with 2416033 documents, 866 features, 243290029 non-zero entries


In [10]:
len(corpus_all)

2416033

In [11]:
ldaseq = ldaseqmodel.LdaSeqModel(corpus=corpus_all, 
                                 id2word=dictionary_all, 
                                 time_slice=yearly_slices, 
                                 passes=2,
                                 num_topics=10,
                                 em_min_iter=1,
                                 em_max_iter=1,
                                 chunksize=12000)

2021-04-25 19:41:07,981 : INFO : using symmetric eta at 0.1
2021-04-25 19:41:07,983 : INFO : using serial LDA version on this node
2021-04-25 19:41:07,985 : INFO : running online (multi-pass) LDA training, 10 topics, 2 passes over the supplied corpus of 2416033 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2021-04-25 19:41:08,069 : INFO : PROGRESS: pass 0, at document #2000/2416033
2021-04-25 19:41:09,371 : INFO : merging changes from 2000 documents into a model of 2416033 documents
2021-04-25 19:41:09,373 : INFO : topic #1 (0.010): 0.012*"china" + 0.012*"united" + 0.011*"states" + 0.009*"high" + 0.007*"cents" + 0.007*"world" + 0.007*"told" + 0.007*"meeting" + 0.007*"japan" + 0.006*"international"
2021-04-25 19:41:09,374 : INFO : topic #7 (0.010): 0.011*"per" + 0.010*"quarter" + 0.009*"company" + 0.007*"half" + 0.007*"second" + 0.006*"inc" + 0.006*"china" + 0.006*"president" + 0.0

2021-04-25 19:41:15,160 : INFO : topic diff=0.202722, rho=0.408248
2021-04-25 19:41:15,210 : INFO : PROGRESS: pass 0, at document #14000/2416033
2021-04-25 19:41:16,221 : INFO : merging changes from 2000 documents into a model of 2416033 documents
2021-04-25 19:41:16,223 : INFO : topic #0 (0.010): 0.019*"quarter" + 0.017*"economist" + 0.016*"federal" + 0.013*"fed" + 0.013*"economists" + 0.013*"inflation" + 0.012*"going" + 0.011*"think" + 0.009*"see" + 0.009*"reserve"
2021-04-25 19:41:16,224 : INFO : topic #5 (0.010): 0.018*"company" + 0.014*"production" + 0.012*"services" + 0.010*"gas" + 0.009*"world" + 0.009*"industry" + 0.009*"told" + 0.008*"added" + 0.008*"years" + 0.008*"output"
2021-04-25 19:41:16,225 : INFO : topic #9 (0.010): 0.020*"central" + 0.019*"exchange" + 0.017*"mark" + 0.015*"currency" + 0.014*"foreign" + 0.013*"fed" + 0.012*"banks" + 0.011*"inflation" + 0.010*"source" + 0.010*"money"
2021-04-25 19:41:16,226 : INFO : topic #7 (0.010): 0.032*"company" + 0.025*"corp" + 0.0

2021-04-25 19:41:22,856 : INFO : topic diff=0.185454, rho=0.288675
2021-04-25 19:41:22,912 : INFO : PROGRESS: pass 0, at document #26000/2416033
2021-04-25 19:41:23,888 : INFO : merging changes from 2000 documents into a model of 2416033 documents
2021-04-25 19:41:23,890 : INFO : topic #9 (0.010): 0.022*"central" + 0.022*"inflation" + 0.022*"fed" + 0.020*"policy" + 0.017*"exchange" + 0.017*"swiss" + 0.017*"federal" + 0.017*"foreign" + 0.017*"banks" + 0.014*"monetary"
2021-04-25 19:41:23,891 : INFO : topic #0 (0.010): 0.020*"fed" + 0.019*"inflation" + 0.019*"federal" + 0.018*"quarter" + 0.018*"economist" + 0.015*"economists" + 0.013*"consumer" + 0.011*"reserve" + 0.011*"going" + 0.011*"treasury"
2021-04-25 19:41:23,893 : INFO : topic #5 (0.010): 0.015*"company" + 0.013*"asia" + 0.013*"services" + 0.013*"world" + 0.013*"industry" + 0.012*"years" + 0.010*"production" + 0.010*"business" + 0.009*"america" + 0.009*"power"
2021-04-25 19:41:23,894 : INFO : topic #6 (0.010): 0.022*"trading" + 0

2021-04-25 19:41:29,438 : INFO : PROGRESS: pass 0, at document #38000/2416033
2021-04-25 19:41:30,495 : INFO : merging changes from 2000 documents into a model of 2416033 documents
2021-04-25 19:41:30,497 : INFO : topic #5 (0.010): 0.015*"world" + 0.013*"industry" + 0.013*"years" + 0.013*"production" + 0.011*"company" + 0.011*"gas" + 0.010*"asia" + 0.010*"america" + 0.009*"investment" + 0.009*"latin"
2021-04-25 19:41:30,498 : INFO : topic #2 (0.010): 0.027*"report" + 0.020*"unemployment" + 0.017*"rose" + 0.016*"average" + 0.016*"jobs" + 0.015*"labor" + 0.014*"department" + 0.014*"increase" + 0.014*"month" + 0.012*"canada"
2021-04-25 19:41:30,499 : INFO : topic #4 (0.010): 0.052*"tax" + 0.027*"budget" + 0.027*"city" + 0.021*"debt" + 0.020*"fund" + 0.018*"value" + 0.018*"local" + 0.017*"general" + 0.014*"public" + 0.014*"issue"
2021-04-25 19:41:30,500 : INFO : topic #7 (0.010): 0.032*"company" + 0.022*"share" + 0.018*"corp" + 0.017*"net" + 0.017*"group" + 0.017*"earnings" + 0.015*"profit

2021-04-25 19:41:37,540 : INFO : topic diff=0.106267, rho=0.204124
2021-04-25 19:41:37,603 : INFO : PROGRESS: pass 0, at document #50000/2416033
2021-04-25 19:41:38,665 : INFO : merging changes from 2000 documents into a model of 2416033 documents
2021-04-25 19:41:38,668 : INFO : topic #3 (0.010): 0.030*"german" + 0.027*"currency" + 0.022*"europe" + 0.022*"japanese" + 0.018*"germany" + 0.010*"currencies" + 0.010*"monetary" + 0.010*"gmt" + 0.009*"late" + 0.009*"high"
2021-04-25 19:41:38,669 : INFO : topic #6 (0.010): 0.021*"stock" + 0.021*"stocks" + 0.020*"trading" + 0.015*"wall" + 0.015*"street" + 0.015*"traders" + 0.015*"dow" + 0.014*"close" + 0.013*"dealers" + 0.012*"futures"
2021-04-25 19:41:38,670 : INFO : topic #1 (0.010): 0.024*"states" + 0.024*"united" + 0.021*"china" + 0.015*"world" + 0.013*"countries" + 0.011*"told" + 0.011*"washington" + 0.011*"president" + 0.010*"korea" + 0.010*"south"
2021-04-25 19:41:38,671 : INFO : topic #2 (0.010): 0.023*"report" + 0.021*"labor" + 0.019*

...

2021-04-25 19:51:28,191 : INFO : topic #3 (0.010): 0.055*"forex" + 0.053*"euro" + 0.044*"politics" + 0.038*"spo" + 0.034*"yen" + 0.029*"fed" + 0.028*"central" + 0.024*"diary" + 0.023*"japan" + 0.023*"inflation"
2021-04-25 19:51:28,192 : INFO : topic diff=0.018468, rho=0.041812
2021-04-25 19:51:28,273 : INFO : PROGRESS: pass 0, at document #1146000/2416033
2021-04-25 19:51:29,007 : INFO : merging changes from 2000 documents into a model of 2416033 documents
2021-04-25 19:51:29,009 : INFO : topic #7 (0.010): 0.049*"call" + 0.046*"corp" + 0.038*"xtra" + 0.033*"editing" + 0.033*"summaries" + 0.026*"holiday" + 0.024*"stock" + 0.021*"earnings" + 0.021*"quarter" + 0.018*"group"
2021-04-25 19:51:29,010 : INFO : topic #5 (0.010): 0.022*"editor" + 0.022*"need" + 0.022*"help" + 0.022*"find" + 0.022*"phone" + 0.021*"minute" + 0.021*"headlines" + 0.021*"stories" + 0.020*"depend" + 0.020*"multimedia"
2021-04-25 19:51:29,012 : INFO : topic #1 (0.010): 0.023*"keywords" + 0.015*"united" + 0.013*"states

2021-04-25 19:51:33,054 : INFO : topic diff=0.017214, rho=0.041595
2021-04-25 19:51:33,135 : INFO : PROGRESS: pass 0, at document #1158000/2416033
2021-04-25 19:51:33,851 : INFO : merging changes from 2000 documents into a model of 2416033 documents
2021-04-25 19:51:33,853 : INFO : topic #9 (0.010): 0.062*"meeting" + 0.054*"annual" + 0.043*"president" + 0.036*"minister" + 0.035*"conference" + 0.033*"gmt" + 0.032*"expected" + 0.028*"washington" + 0.028*"federal" + 0.026*"prime"
2021-04-25 19:51:33,854 : INFO : topic #6 (0.010): 0.078*"nasdaq" + 0.074*"dow" + 0.066*"jones" + 0.060*"dji" + 0.055*"nikkei" + 0.052*"res" + 0.043*"currency" + 0.040*"profit" + 0.034*"results" + 0.033*"indicators"
2021-04-25 19:51:33,855 : INFO : topic #4 (0.010): 0.037*"int" + 0.033*"rating" + 0.024*"tax" + 0.024*"bln" + 0.022*"fin" + 0.019*"income" + 0.019*"fiscal" + 0.018*"ratings" + 0.018*"state" + 0.017*"city"
2021-04-25 19:51:33,857 : INFO : topic #8 (0.010): 0.051*"click" + 0.050*"eci" + 0.041*"euribor" 

2021-04-25 19:51:42,038 : INFO : topic #5 (0.010): 0.022*"editor" + 0.022*"need" + 0.022*"help" + 0.022*"find" + 0.022*"phone" + 0.021*"minute" + 0.021*"headlines" + 0.021*"stories" + 0.020*"depend" + 0.020*"plus"
2021-04-25 19:51:42,039 : INFO : topic #1 (0.010): 0.022*"keywords" + 0.014*"united" + 0.013*"states" + 0.011*"obama" + 0.010*"china" + 0.009*"also" + 0.009*"one" + 0.009*"president" + 0.009*"nsp" + 0.008*"gas"
2021-04-25 19:51:42,040 : INFO : topic #7 (0.010): 0.043*"xtra" + 0.039*"corp" + 0.038*"summaries" + 0.037*"editing" + 0.036*"call" + 0.027*"stock" + 0.023*"holiday" + 0.021*"earnings" + 0.018*"quarter" + 0.018*"share"
2021-04-25 19:51:42,041 : INFO : topic #6 (0.010): 0.079*"nasdaq" + 0.077*"dow" + 0.067*"jones" + 0.061*"dji" + 0.057*"nikkei" + 0.053*"res" + 0.044*"currency" + 0.037*"profit" + 0.033*"results" + 0.033*"indicators"
2021-04-25 19:51:42,042 : INFO : topic #4 (0.010): 0.038*"int" + 0.032*"rating" + 0.024*"tax" + 0.024*"bln" + 0.023*"fin" + 0.020*"income" +

2021-04-25 19:51:54,850 : INFO : topic #2 (0.010): 0.031*"report" + 0.030*"per" + 0.027*"price" + 0.024*"demand" + 0.023*"sales" + 0.022*"chicago" + 0.018*"production" + 0.017*"average" + 0.017*"export" + 0.015*"crude"
2021-04-25 19:51:54,851 : INFO : topic #1 (0.010): 0.022*"keywords" + 0.014*"united" + 0.014*"obama" + 0.013*"states" + 0.010*"china" + 0.010*"also" + 0.009*"one" + 0.009*"nsp" + 0.009*"president" + 0.008*"two"
2021-04-25 19:51:54,853 : INFO : topic #8 (0.010): 0.051*"eci" + 0.048*"click" + 0.042*"euribor" + 0.042*"frx" + 0.042*"usbmk" + 0.042*"efx" + 0.038*"dbt" + 0.038*"mrg" + 0.037*"pct" + 0.036*"equ"
2021-04-25 19:51:54,854 : INFO : topic #3 (0.010): 0.055*"forex" + 0.050*"euro" + 0.044*"politics" + 0.039*"spo" + 0.031*"yen" + 0.030*"inflation" + 0.029*"fed" + 0.028*"central" + 0.024*"diary" + 0.023*"japan"
2021-04-25 19:51:54,855 : INFO : topic diff=0.018511, rho=0.041135
2021-04-25 19:51:55,026 : INFO : PROGRESS: pass 0, at document #1184000/2416033
2021-04-25 19:5

KeyboardInterrupt: 